In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

import requests
import json

random_state = 42

In [2]:
raw_data = pd.merge(pd.read_csv('../train_transaction.csv'), pd.read_csv('../train_identity.csv'),
                    left_on='TransactionID', right_on='TransactionID', how='left')


Подготовим данные для запроса, по которому выполняется обучение модели.

In [3]:
df_req_train = raw_data.sample(1000, random_state=random_state)
df_req_train.reset_index(drop=True, inplace=True)

Из тестового датасета возьмем 50 мошеннических и 50 немошеннических транзакций для тестирования запросов, по которым выплняется предсказание.

In [4]:
X_train, X_test = train_test_split(raw_data, test_size=0.2, random_state=random_state)

X_test.reset_index(drop=True, inplace=True)
y_test = X_test['isFraud']

del raw_data
del X_train

In [5]:
df_req_fraud = X_test[y_test==1].sample(50, random_state=random_state)
df_req_fraud.reset_index(drop=True, inplace=True)

df_req_not_fraud = X_test[y_test==0].sample(50, random_state=random_state)
df_req_not_fraud.reset_index(drop=True, inplace=True)

In [6]:
del X_test

### Запрос для одного объекта

Для каждой транзакции выполним отдельный запрос

Результаты запросов: 0 - транзакция не мошенническая, 1 - транзакция мошенническая

In [10]:
url = 'http://127.0.0.1:8000/predict_one_transaction'
response = requests.post(url, data=json.dumps(df_req_fraud.loc[2].to_dict()), headers={"Content-Type": "application/json"})
print(response.status_code)
print(response.json())

200
{'task_id': 'f732f134-af58-4299-8802-2a66eaba3159'}


In [11]:
task_id = response.json()['task_id']
url = 'http://127.0.0.1:8000/task/' + task_id
task_info_resp = requests.get(url)

print('response status:', task_info_resp.status_code)
print()
res = task_info_resp.json()
if res['task_status'] != 'SUCCESS':
    print('task_status =', res['task_status'])
    print('task_id =', res['task_id'])
else:
    print('task_id =', res['task_id'])
    print('task_status =', res['task_status'])
    print('task_result =', res['task_result'])

response status: 200

task_id = f732f134-af58-4299-8802-2a66eaba3159
task_status = SUCCESS
task_result = [1]


### Запрос для списка транзакций

In [22]:
url_items = 'http://127.0.0.1:8000/predict_transactions'
lst = []
for i in range(df_req_fraud.shape[0]):
    lst.append(df_req_fraud.loc[i].to_dict())
response_items = requests.post(url_items, data=json.dumps(lst), headers={"Content-Type": "application/json"})
print(response_items.status_code)
print(response_items.json())

200
{'task_id': 'e3292cc2-4b98-46d5-8efe-15cc7698914a'}


In [24]:
task_id = response_items.json()['task_id']
url = 'http://127.0.0.1:8000/task/' + task_id
task_info_resp = requests.get(url)

print('response status:', task_info_resp.status_code)
print()
res = task_info_resp.json()
if res['task_status'] != 'SUCCESS':
    print('task_status =', res['task_status'])
    print('task_id =', res['task_id'])
else:
    print('task_id =', res['task_id'])
    print('task_status =', res['task_status'])
    print('task_result =', res['task_result'])
    print('detected', sum(res['task_result']), 'fraud of', df_req_fraud.shape[0])

response status: 200

task_id = e3292cc2-4b98-46d5-8efe-15cc7698914a
task_status = SUCCESS
task_result = [0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0]
detected 16 fraud of 50


### Запрос для обучения модели

In [19]:
url_train = 'http://127.0.0.1:8000/train_input_data'
response_train = requests.post(url_train, data=json.dumps(df_req_train.to_dict(orient='list')), headers={"Content-Type": "application/json"})
response_train.json()

{'task_id': '9d425ea0-1815-4ea3-a543-fb04b90ed613'}

In [21]:
task_id = response_train.json()['task_id']
url = 'http://127.0.0.1:8000/task/' + task_id
task_info_resp = requests.get(url)

print('response status:', task_info_resp.status_code)
print()
res = task_info_resp.json()
if res['task_status'] != 'SUCCESS':
    print('task_status =', res['task_status'])
    print('task_id =', res['task_id'])
else:
    print('task_id =', res['task_id'])
    print('task_status =', res['task_status'])
    print('task_result =', res['task_result'])

response status: 200

task_id = 9d425ea0-1815-4ea3-a543-fb04b90ed613
task_status = SUCCESS
task_result = True


### Добавить транзакцию в БД

In [18]:
url_add_trn = 'http://127.0.0.1:8000/add_transaction'
for i in range(0, 1000, 100):
    response = requests.post(url_add_trn, data=json.dumps(df_req_train.loc[i].to_dict()), headers={"Content-Type": "application/json"})
    print(response.json())

{'id': 3457624, 'status': 'already in db'}
{'id': 3370779, 'status': 'already in db'}
{'id': 3205104, 'status': 'already in db'}
{'id': 3284271, 'status': 'already in db'}
{'id': 3292889, 'status': 'already in db'}
{'id': 3416674, 'status': 'already in db'}
{'id': 3257408, 'status': 'already in db'}
{'id': 3222300, 'status': 'already in db'}
{'id': 3382253, 'status': 'already in db'}
{'id': 3206312, 'status': 'already in db'}


### Запрос для тестирования панели 'Total Exception' дашборда 'FastAPI Service' в Grafana.
По этому запросу сервис ничего не делает. Только бросает исключение

In [60]:
url_raise_exception = 'http://127.0.0.1:8000/raise_exception'
for i in range(50):
    resp_raise_exception = requests.get(url_raise_exception)
resp_raise_exception.status_code

500

### Grafana

Посмотрим на дашборд 'FastAPI Service'

Grafana http://localhost:3000/

<center>
<img src="grafana_dashboard.JPG"> 
</center>